In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from imblearn.combine import SMOTETomek



from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import expon
from sklearn.model_selection import RandomizedSearchCV



from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
train_data = pd.read_csv("../2_Data_Clean/hr/train_data.csv")

X = train_data.drop(["Class"], axis = 1)
y = train_data['Class']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, stratify = y, random_state = 7)

sampler = SMOTETomek()
X_resample, y_resample = sampler.fit_sample(X_train, y_train)

In [4]:
pipe = make_pipeline(StandardScaler(),RandomForestClassifier())
pipe

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [6]:
from scipy.stats import randint

param_grid = {
    'randomforestclassifier__criterion':['gini', 'entropy'],
    'randomforestclassifier__n_estimators':randint(100,1000)
}

random_cv = RandomizedSearchCV(estimator = pipe,param_distributions = param_grid, cv =10, error_score = 0,
                               n_iter = 10 , scoring = 'f1',return_train_score=True, n_jobs = 2)

In [ ]:
y_pred = random_cv.predict(X_test)

cm = confusion_matrix(y_true = y_test, y_pred = y_pred)

sns.heatmap(cm, annot = True)
plt.xlabel("Predicted value")
plt.ylabel("True value")